# Quantum Machine Learning with PennyLane

### Sergio Andrés Cotrino Sandoval & José Luis Falla León
### Friday, March 5th, 2021
### Chaos & Complexity Group
### Universidad Nacional de Colombia

## Table of Contents
- Introduction to PennyLane
- PennyLane Features
    - Continuous-variable Architecture
    - Qubit Architecture
- Example 1: Optimizing a Quantum Optical Neural Network
- Example 2: Variational Quantum Classifier
- Crossover: PennyLane + Qiskit
- References

# Introduction to PennyLane
- Cross-platform, open-source, Python library for differentiable programming of quantum computers.
- Facilitates the optimization of both quantum and hybrid quantum-classical algorithms.
- PennyLane can, in principle, be used with any gate-based quantum computing platform as a backend, including both qubit and continuous-variable architectures.

# Continuous Variables

# Optimizing a quantum optical neural network

This is a review of the tutorial that can be found on https://pennylane.ai/qml/demos/qonn.html. This Quantum Optica Neural Network (QONN) is based on Fock states, and attempts to apply neural networks and deep learning theory to the quantum case, using quantum data and architecture, and a classical optimization process.

<MORE BACKGROUND>

In [1]:
# Import PennyLane and the numpy wrapper
# Wrapper needed to perform some functions of np in the context of PennyLane
import pennylane as qml
from pennylane import numpy as np

A Strawberry Fields simulator is created with as many quantun modes as the quantum-optical neural network is to have. Also, as Fock states are used, a cutoff dimension is needed. In this case, the cutoff will be the same as the number of quantum modes.

In [3]:
dev = qml.device("strawberryfields.fock", wires=4, cutoff_dim=4)

To create the QONN, layers are used. These consist of a linear interferometer and a non-linear Kerr interaction layer. Both layers are aplied to all modes. More information regarding this layout can be found on <Reck et al. (1994)>. The main idea is that a given sequence of beam splitter transformations can execute any discrete finite-dimensional unitary operation.

In [4]:
def layer(theta, phi, wires):
    M = len(wires)
    phi_nonlinear = np.pi / 2

    # Created the interferometer using a template
    qml.templates.Interferometer(
        theta, phi, np.zeros(M), wires=wires, mesh="triangular",
    )

    # Creater the non-linear Kerr interaction layer
    # The parameter for nonlinear interaction is constant
    for i in wires:
        qml.Kerr(phi_nonlinear, wires=i)

In [6]:
# The full QONN is buid as a circuit
# The parameters to be optimized are contained in var
# Each element in var is a list of parameters theta and phi for a specific layer.

# The decorator "@qml.qnode(dev)" indicates that the decorated function contains a quantum circuit bounded to a compatible device already created

@qml.qnode(dev)
def quantum_neural_net(var, x):
    wires = list(range(len(x)))

    # Encode input x into a sequence of quantum fock states
    for i in wires:
        qml.FockState(x[i], wires=i)

    # "layer" subcircuits
    for i, v in enumerate(var):
        layer(v[: len(v) // 2], v[len(v) // 2 :], wires)

    # At the end, the number operator is used to obtain the mean photon number in each mode
    return [qml.expval(qml.NumberOperator(w)) for w in wires]

As a cost function, a helper function is used to aid in the calculation of the normalized square loss of two vectors. If zero, both parameters are equal. If 1, they are fully orthogonal.

In [7]:
# User for several pairs of verctors
def square_loss(labels, predictions):
    term = 0
    for l, p in zip(labels, predictions):
        # Normalization of each vector
        lnorm = l / np.linalg.norm(l)
        pnorm = p / np.linalg.norm(p)

        # The makes the inner product for a pair of vectors
        term = term + np.abs(np.dot(lnorm, pnorm.T)) ** 2

    # Returns the loss: 1 mminus the average of the inner products
    return 1 - term / len(labels)

Then, the cost function is defined. It uses the outputs from the QONN (predictions) for each input (data_inputs) and the calculates the square loss between the predictions ans the true outputs (labels).

In [9]:
def cost(var, data_input, labels):
    # Gets the predictions for data_input and then calculates the loss
    predictions = np.array([quantum_neural_net(var, x) for x in data_input])
    sl = square_loss(labels, predictions)

    return sl

## Optimizing the CNOT gate

Now, the network can be optimized by choosing  a function from imput to output states.Four modes were chosen to use a gate of two qubits. In this case, the network is trained as a CNOT gate.

Inputs and labels are then defined.

In [10]:
# Define the CNOT input-output states (dual-rail encoding) and initialize
# them as non-differentiable.

X = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 1, 0],
              [0, 1, 0, 1]], requires_grad=False)

Y = np.array([[1, 0, 1, 0],
              [1, 0, 0, 1],
              [0, 1, 0, 1],
              [0, 1, 1, 0]], requires_grad=False)

Then:

* Define the number of layers
* Calculate and get the corresponding number of initial parameters to use. To better undestanding of that, it would be useful to check the definition of the interferometer defined in PennyLane.

In the case bellow, with 4 modes, there are 4*(4-1)=12 variables per layer. This is the pposible number of interactions of the interferometes with a triangular layout.

In [11]:
num_layers = 2
M = len(X[0])
num_variables_per_layer = M * (M - 1)

var_init = (4 * np.random.rand(num_layers, num_variables_per_layer) - 2) * np.pi
print(var_init)

[[ 0.44649261  0.41210471 -4.74250452  5.83428451 -2.80530642  3.36008672
   3.61030206  5.83685284 -2.64962557  1.15000548 -5.93954918  4.05653285]
 [ 5.41041372 -4.47735875  0.43822647  2.19782286 -3.81815685  1.91523848
   0.90973825 -2.1355572  -3.85285685 -5.09985053  5.72977364  2.70602782]]


In [ ]:
from pennylane.optimize import AdamOptimizer

opt = AdamOptimizer(0.01, beta1=0.9, beta2=0.999)

var = var_init
for it in range(200):
    var = opt.step(lambda v: cost(v, X, Y), var)

    if (it+1) % 20 == 0:
        print(f"Iter: {it+1:5d} | Cost: {cost(var, X, Y):0.7f} ")

In [12]:
print(f"The optimized parameters (layers, parameters):\n {var}\n")

Y_pred = np.array([quantum_neural_net(var, x) for x in X])
for i, x in enumerate(X):
    print(f"{x} --> {Y_pred[i].round(2)}, should be {Y[i]}")

NameError: name 'var' is not defined

In [13]:
# Finally, we can draw the resulting circuit for a given input
quantum_neural_net(var_init, X[0])
print(quantum_neural_net.draw())

 0: ──|1⟩──────────────────────────────────────╭BS(-4.74, -2.65)───R(0)───────────────Kerr(1.57)──────────────────────────────────────────────────────────────────╭BS(0.438, -3.85)───R(0)──────────────Kerr(1.57)──────────────────────────────┤ ⟨n⟩ 
 1: ──|0⟩────────────────────╭BS(0.412, 5.84)──╰BS(-4.74, -2.65)──╭BS(-2.81, -5.94)───R(0)────────────Kerr(1.57)───────────────────────────────╭BS(-4.48, -2.14)──╰BS(0.438, -3.85)──╭BS(-3.82, 5.73)───R(0)────────────Kerr(1.57)──────────────┤ ⟨n⟩ 
 2: ──|1⟩──╭BS(0.446, 3.61)──╰BS(0.412, 5.84)──╭BS(5.83, 1.15)────╰BS(-2.81, -5.94)──╭BS(3.36, 4.06)──R(0)────────Kerr(1.57)──╭BS(5.41, 0.91)──╰BS(-4.48, -2.14)──╭BS(2.2, -5.1)─────╰BS(-3.82, 5.73)──╭BS(1.92, 2.71)──R(0)────────Kerr(1.57)──┤ ⟨n⟩ 
 3: ──|0⟩──╰BS(0.446, 3.61)────────────────────╰BS(5.83, 1.15)───────────────────────╰BS(3.36, 4.06)──R(0)────────Kerr(1.57)──╰BS(5.41, 0.91)─────────────────────╰BS(2.2, -5.1)───────────────────────╰BS(1.92, 2.71)──R(0)────────Kerr(1.57)──┤ ⟨n⟩ 



# Variational Quantum Classifier
* A circuit can be trained from labelled data to classify new data samples.
* VQC can reproduce the parity function:
$$ f : x \in \{0, 1\}^{\otimes n} \longrightarrow y = \begin{cases} 1 & \text{if uneven number of ones in x} \\ 0 & \text{otherwise} \end{cases}$$

In [1]:
import pennylane as qml
import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [2]:
# Create quantum device with four wires
dev = qml.device("default.qubit", wires=4)

In [3]:
def layer(W):
    
    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)
    
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])